## Simulation
### Variables for simulation
* Number of doses
* TTL
* Dose-toxicity model
* Dose-toxicity sketon
* Method of inference
* Decision rules
* Samlpe size and cohort size
* Safety modification
* Stoping rules

In [ ]:
import pymc as pm
import numpy as np
from tqdm.notebook import tqdm
from scipy.stats import bernoulli
import logging

In [ ]:
# Define a two-parameter logistic regression model.
# The parameters are the intercept (b1) and the slope (b2).
def two_para_logit(d, b1, b2):
    return np.exp(b1 + b2*d) / (1 + np.exp(b1 + b2*d))

In [ ]:
# Define the dose selection function.
def bcrm_dose_selection(p_prior, sample_size=36, ttl=1/3, d_levels=np.array([0.5, 1, 3, 5, 6]), b1_prior=0, b2_prior=1):
    # Generate a sample of patients.
    patient_dose = np.random.choice(d_levels, size=sample_size)
    patient_toxicity = bernoulli.rvs(p_prior[np.searchsorted(d_levels, patient_dose)])
    dose_patient_num = np.array([np.sum(patient_dose == d) for d in d_levels])

    # likelihood
    with pm.Model() as model:
        # prior
        b1 = pm.Normal('b1', mu=b1_prior, tau=10)
        b2 = pm.Exponential('b2', lam=b2_prior)
        
        # likelihood
        p_toxicity = pm.Deterministic('p_toxicity', two_para_logit(patient_dose, b1, b2))
        # print(patient_toxicity)
        likelihood = pm.Bernoulli('toxicity', p=p_toxicity, observed=patient_toxicity)

        # sample
        logging.getLogger('pymc').setLevel(logging.ERROR)
        trace = pm.sample(1000, tune=500, cores=4, chains=2)

    # Find the MTD.
    b1_posterior = np.mean(trace['posterior']['b1']).item()
    b2_posterior = np.mean(trace['posterior']['b2']).item()
    # print('b1_posterior: ', b1_posterior)
    # print('b2_posterior: ', b2_posterior)
    p_posterior = two_para_logit(d_levels, b1_posterior, b2_posterior)
    mtd = d_levels[np.argmin(np.abs(p_posterior - ttl))]
    return mtd, dose_patient_num

In [ ]:
# Define the simulation function.
def sim_bcrm(p_prior, N=1000, d_levels=np.array([0.5, 1, 3, 5, 6])):
    mtds = np.zeros((N, len(d_levels)))
    patient_doses = np.zeros((N, len(d_levels)))
    for i in tqdm(range(N)):
        mtd, patients = bcrm_dose_selection(p_prior)
        mtds[i, :] = (mtd == d_levels)
        patient_doses[i, :] = patients
    return mtds, patient_doses

In [ ]:
# Scenario 1
p_prior_1 = np.array([0.25, 0.3, 0.5, 0.6, 0.7])
mtd_1, patient_doses_1 = sim_bcrm(p_prior_1)

In [ ]:
print(np.mean(mtd_1, axis=0))
print(np.mean(patient_doses_1, axis=0))